# Multiclass SVM 구현

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

#IRIS 데이터 로드
iris =  sns.load_dataset('iris') 
X= iris.iloc[:,:4] #학습할데이터
y = iris.iloc[:,-1] #타겟
print(y)

0         setosa
1         setosa
2         setosa
3         setosa
4         setosa
         ...    
145    virginica
146    virginica
147    virginica
148    virginica
149    virginica
Name: species, Length: 150, dtype: object


In [2]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=48)

In [3]:
def standardization(train, test):
    scaler = StandardScaler()
    train = scaler.fit_transform(train)
    test = scaler.transform(test)
    return train, test

X_train, X_test = standardization(X_train, X_test)

In [4]:
X_train

array([[ 0.78522493,  0.32015325,  0.77221097,  1.04726529],
       [-0.26563371, -1.29989934,  0.0982814 , -0.11996537],
       [ 0.43493872,  0.78302542,  0.94069336,  1.43634218],
       [-0.84944407,  0.78302542, -1.24957775, -1.28719604],
       [-0.38239578, -1.7627715 ,  0.15444219,  0.13941922],
       [ 0.55170079, -0.374155  ,  1.05301496,  0.7878807 ],
       [ 0.31817664, -0.14271892,  0.65988937,  0.7878807 ],
       [ 0.20141457, -0.374155  ,  0.43524618,  0.39880381],
       [-1.66677857, -0.14271892, -1.36189934, -1.28719604],
       [-0.14887164, -0.60559109,  0.21060299,  0.13941922],
       [-0.14887164, -1.06846325, -0.12636179, -0.24965767],
       [ 0.31817664, -0.60559109,  0.15444219,  0.13941922],
       [ 0.66846286, -0.83702717,  0.88453256,  0.91757299],
       [ 0.0846525 , -0.14271892,  0.77221097,  0.7878807 ],
       [-0.49915786, -0.14271892,  0.43524618,  0.39880381],
       [-0.26563371, -0.60559109,  0.65988937,  1.04726529],
       [ 2.18636979,  1.

In [5]:
X_test

array([[-0.14887164, -0.374155  ,  0.26676379,  0.13941922],
       [ 0.31817664, -0.60559109,  0.54756778,  0.00972692],
       [ 0.31817664, -1.06846325,  1.05301496,  0.26911151],
       [-1.5500165 , -1.7627715 , -1.36189934, -1.15750374],
       [ 0.0846525 ,  0.32015325,  0.60372857,  0.7878807 ],
       [ 0.78522493, -0.14271892,  0.99685416,  0.7878807 ],
       [-0.84944407,  1.70876975, -1.24957775, -1.15750374],
       [ 0.20141457, -0.14271892,  0.60372857,  0.7878807 ],
       [-0.38239578,  2.63451409, -1.30573855, -1.28719604],
       [-0.38239578, -1.29989934,  0.15444219,  0.13941922],
       [ 0.66846286,  0.08871717,  0.99685416,  0.7878807 ],
       [-0.38239578,  1.0144615 , -1.36189934, -1.28719604],
       [-0.49915786,  0.78302542, -1.13725615, -1.28719604],
       [ 0.43493872, -0.60559109,  0.60372857,  0.7878807 ],
       [ 0.55170079, -1.7627715 ,  0.37908538,  0.13941922],
       [ 0.55170079,  0.55158933,  0.54756778,  0.52849611],
       [-1.19973028,  0.

In [6]:
# one-hot encoding
y_train = pd.get_dummies(y_train)

In [7]:
y_train.head()

,setosa,versicolor,virginica
110,0,0,1
69,0,1,0
148,0,0,1
39,1,0,0
53,0,1,0


## One vs Rest SVM

전체 class 수가 $M$개라고 하면 $i$번째 class 부류와 $i$클래스를 제외한 나머지 $M-1$개의 클래스가 속하는 클래스로 이진화 시켜 분류기를 만들고, $M$번만큼 반복함. 총 $M$개의 이진분류기가 만들어짐. 즉, $i class$에 속하는 샘플을 +1 라벨을 붙이고 나머지 샘플에 -1 라벨을 붙여 훈련집합을 만듦. svm의 결정 초평면도 $M$개가 만들어지는데 $j$번째 초평면을 $d(j)$라 할 수 있음. 실제 Test할 때에는 $M$개의 초평면에 모두 test를 하게 되는데 $m$번의 분류에서 1가지만 양수를 출력하고 나머지는 모두 음수를 출력한다면 문제될 것이 전혀 없지만, 항상 그렇게 된다는 보장은 없음. 따라서 $m$번의 test 중 가장 큰 $d(j)$를 갖는 것을 예측된 class로 함.

In [12]:
class One_vs_Rest_SVM:
    def __init__(self, n_classes=3):
        self.n_classes = n_classes
        self.clfs = []
        self.y_pred = [ ]
        
    # encoding된 y_train을 class 개수만큼 classifier 반복
    def fit(self, X_train, y_train, C=5, gamma=5):
        y_encoded = y_train
        
        for i in range(self.n_classes):
            clf = SVC(kernel='rbf', C=C, gamma=gamma)
            clf.fit(X_train, y_encoded.iloc[:,i])
            self.clfs.append(clf)
    
    # 각각의 classifier에서 나온 결과를 바탕으로 투표 진행        
    def predict(self, X_test):
        vote = np.zeros((len(X_test), 3), dtype=int)
        size = X_test.shape[0]
        
        for i in range(size):
            # 해당 class에 속하는 샘플을 +1만큼, 나머지 샘플에는 -1만큼 투표
            if self.clfs[0].predict(X_test)[i] == 1:
                vote[i][0] += 1
                vote[i][1] -= 1
                vote[i][2] -= 1
            elif self.clfs[1].predict(X_test)[i] == 1:
                vote[i][0] -= 1
                vote[i][1] += 1
                vote[i][2] -= 1
            elif self.clfs[2].predict(X_test)[i] == 1:
                vote[i][0] -= 1
                vote[i][1] -= 1
                vote[i][2] += 1
                
            # 투표한 값 중에서 가장 큰 값의 인덱스를 test label에 넣음
            self.y_pred.append(np.argmax(vote[i]))
            
            # 경우의 수
            # 1. 한 분류기의 투표 결과가 양수, 나머지는 음수인 경우
            # 2. 세 분류기의 투표 결과가 모두 0으로 같은 경우
            # 3. 두 분류기의 투표 결과가 양수로 같은 경우
            
            # 2번째, 동점일 경우 decision_function의 값이 가장 큰 경우를 test label에 넣음
            if (np.sign(self.clfs[0].decision_function(X_test)[i]) == np.sign(self.clfs[1].decision_function(X_test)[i])) \
            and (np.sign(self.clfs[1].decision_function(X_test)[i]) == np.sign(self.clfs[2].decision_function(X_test)[i])) :
                self.y_pred[i] = np.argmax([self.clfs[0].decision_function(X_test)[i], self.clfs[1].decision_function(X_test)[i], self.clfs[2].decision_function(X_test)[i]])
                
            # 3번째, 두 분류기의 투표 결과가 양수로 같을 경우 decision_function의 값이 가장 큰 경우를 test label에 넣음
            elif (vote[i][0] == vote[i][1]) and vote[i][0] > 0 and vote[i][1] > 0:
                self.y_pred[i] = np.argmax([self.clfs[0].decision_function(X_test)[i], self.clfs[1].decision_function(X_test)[i]])
            elif (vote[i][0] == vote[i][2]) and vote[i][0] > 0 and vote[i][2] > 0 :
                self.y_pred[i] = np.argmax([self.clfs[0].decision_function(X_test)[i], self.clfs[2].decision_function(X_test)[i]])
            elif (vote[i][1] == vote[i][2]) and vote[i][1] > 0 and vote[i][2] > 0:
                self.y_pred[i] = np.argmax([self.clfs[1].decision_function(X_test)[i], self.clfs[2].decision_function(X_test)[i]])
            
        # test를 진행하기 위해 0,1,2로 되어있던 데이터를 다시 문자 label로 변환
        self.y_pred = pd.DataFrame(self.y_pred).replace({0:'setosa', 1:'versicolor', 2:'virginica'})
        return self.y_pred
    
    # accuracy 확인
    def evaluate(self, y_test):
        print('Accuracy : {: .5f}'.format(accuracy_score(y_test, self.y_pred)))

In [13]:
onevsrest = One_vs_Rest_SVM()
onevsrest.fit(X_train, y_train)

In [14]:
y_pred_rest = onevsrest.predict(X_test)
y_pred_rest

,0
0,versicolor
1,versicolor
2,versicolor
3,virginica
4,virginica
5,virginica
6,setosa
7,virginica
8,setosa
9,versicolor


In [15]:
onevsrest.evaluate(y_test)

Accuracy :  0.86667


one vs rest svm의 경우 투표의 개념을 도입할 필요 없이 서로 얼마나 멀리 떨어져 있는지를 나타내는 decision function값만 비교해줘도 됨.

결과 패턴
* 정상적으로 예측 결과가 나왔을 때(한 곳에서만 값이 1) : 한 곳에서만 Decision_function값이 양수
* 2개 이상의 클래스로 예측되었을 때(두 개 이상의 1) : 복수의 클래스 중 더 특성을 잘 나타내는 클래스의 Decision_function값이 클 것임.
* 어떠한 클래스로도 예측되지 않았을 때(모두 0) : 그나마 가장 가까운 클래스의 Decision_function값이 음수이며 절대값이 가장 작을 것

In [16]:
class One_vs_Rest_SVM:
    def __init__(self, n_classes=3):
        self.n_classes = n_classes
        self.clfs = []
        self.y_pred = [ ]
        
    # encoding된 y_train을 class 개수만큼 classifier 반복
    def fit(self, X_train, y_train, C=5, gamma=5):
        y_encoded = y_train
        
        for i in range(self.n_classes):
            clf = SVC(kernel='rbf', C=C, gamma=gamma)
            clf.fit(X_train, y_encoded.iloc[:,i])
            self.clfs.append(clf)
    
    # 각각의 classifier에서 나온 결과를 decision function으로 비교
    def predict(self, X_test):
        vote = np.zeros((len(X_test), 3), dtype = int)
        size = X_test.shape[0]
        
        for i in range(size):
            self.y_pred.append(np.argmax([self.clfs[0].decision_function(X_test)[i], self.clfs[1].decision_function(X_test)[i], self.clfs[2].decision_function(X_test)[i]]))
        
        # test를 진행하기 위해 0,1,2로 되어있던 데이터를 다시 문자 label로 변환
        self.y_pred = pd.DataFrame(self.y_pred).replace({0:'setosa', 1:'versicolor', 2:'virginica'})
        return self.y_pred
    
    # accuracy 확인
    def evaluate(self, y_test):
        print('Accuracy : {: .5f}'.format(accuracy_score(y_test, self.y_pred)))

In [17]:
onevsrest = One_vs_Rest_SVM()
onevsrest.fit(X_train, y_train)
y_pred_rest = onevsrest.predict(X_test)
onevsrest.evaluate(y_test)

Accuracy :  0.86667


## 라이브러리 제공 Multiclass SVM

In [18]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X, y, test_size=0.2, random_state=48)

scaler = StandardScaler()
X_train1 = scaler.fit_transform(X_train1)
X_test1 = scaler.transform(X_test1)

svm_lib = SVC(kernel='rbf', C=5, gamma=5)
svm_lib.fit(X_train1, y_train1)
y_pred = svm_lib.predict(X_test1)

accuracy_score(y_test1, y_pred)

0.8666666666666667

같은 성능을 보임.